# Step 1: Load data

In [1]:
import pandas as pd
import numpy as np

In [28]:
rec1data = pd.read_csv("../data/processed/rec1data.csv")
rec2data = pd.read_csv("../data/processed/rec2data.csv")
psychdata = pd.read_csv("../data/processed/psychdata.csv")

# Step 2: Clean data

In [29]:
# Incorrect data
psychdata["date_contact"][3597] = None
psychdata["date_term"][3597] = None

<ipython-input-29-4450631df2b6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psychdata["date_contact"][3597] = None
<ipython-input-29-4450631df2b6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psychdata["date_term"][3597] = None


In [30]:
# Convert dates to usable format
psychdata["date_contact"] = psychdata["date_contact"] + 19000000
psychdata["date_term"] = psychdata["date_term"] + 19000000

In [31]:
# Remove unneeded columns
psychdata = psychdata[["id", "contact", "date_contact", "diagnosis", "service", "date_term", "reason_term"]]
del rec1data["rid"]
del rec2data["rid"]

In [44]:
# Remove unneeded rows from matched non-prisoner cases
rec1data = rec1data[rec1data["crime01"] > 0]
rec2data = rec2data.loc[range(616),:]

In [47]:
# Merge rec1 and rec2 data frames
comb_data = rec1data.merge(rec2data, on = "id", how = "outer")
del comb_data["Unnamed: 0_x"]

In [50]:
# Merge with psychdata, which will delete rows from non-prisoner cases
dta = comb_data.merge(psychdata, on = "id", how = "left")
dta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3479 entries, 0 to 3478
Data columns (total 58 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                3479 non-null   int64  
 1   sex               3479 non-null   int64  
 2   birth             3479 non-null   int64  
 3   race              3479 non-null   int64  
 4   educ              3479 non-null   int64  
 5   marital           3479 non-null   int64  
 6   children          3479 non-null   int64  
 7   occupat           3479 non-null   int64  
 8   arrests           3479 non-null   int64  
 9   jail              3479 non-null   int64  
 10  crime01           3479 non-null   int64  
 11  crime02           3479 non-null   int64  
 12  crime03           3479 non-null   int64  
 13  crime04           3479 non-null   int64  
 14  crime05           3479 non-null   int64  
 15  crime06           3479 non-null   int64  
 16  crime07           3479 non-null   int64  


# Step 3: Process data

In [ ]:
# Create length of treatment variable
dta["date_contact"] = pd.to_datetime(dta["date_contact"], format='%Y%m%d', errors = "coerce")
dta["date_term"] = pd.to_datetime(dta["date_term"], format='%Y%m%d', errors = "coerce")
dta["length_treat"] = (dta["date_term"] - dta["date_contact"]).dt.days
dta["length_treat"].describe()

# Step 4: Wrangle data

I had to do this step earlier to remove the rows that I'm not using.